In [1]:
from pyramid.arima import auto_arima
import pandas as pd
import numpy as np

D:\Programs\Anaconda\lib\site-packages\pyramid\__init__.py:68: UserWarning: 
    The 'pyramid' package will be migrating to a new namespace beginning in 
    version 1.0.0: 'pmdarima'. This is due to a package name collision with the
    Pyramid web framework. For more information, see Issue #34:
    
        https://github.com/tgsmith61591/pyramid/issues/34
        
    The package will subsequently be installable via the name 'pmdarima'; the
    only functional change to the user will be the import name. All imports
    from 'pyramid' will change to 'pmdarima'.
    
  """, UserWarning)


In [2]:
df = pd.read_csv("C:/Users/Desktop/Desktop/Dissertation/snp500/stock_dfs/IBM.csv")

In [3]:
#Setting index as Date and create new dataframe to use as target variable for Adj Close
df['Date'] = pd.to_datetime(df.Date,format='%Y-%m-%d')
df.index = df['Date']

In [4]:
data = df.sort_index(ascending=True, axis=0)

In [5]:
#current shape of an array
data.shape

(4800, 7)

In [6]:
# First 5 rows of IBM 
data.head()

,Date,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,,
2000-01-03,2000-01-03,116.0000,111.875,112.4375,116.0000,10347700.0,80.076149
2000-01-04,2000-01-04,114.5000,110.875,114.0000,112.0625,8227800.0,77.358032
2000-01-05,2000-01-05,119.7500,112.125,112.9375,116.0000,12733200.0,80.076149
2000-01-06,2000-01-06,118.9375,113.500,118.0000,114.0000,7971900.0,78.695503
2000-01-07,2000-01-07,117.9375,110.625,117.2500,113.5000,11856700.0,78.350372


In [7]:
#split into train and validation
train = data[:4750]
valid = data[4750:]

In [8]:
train.shape, valid.shape

((4750, 7), (50, 7))

In [9]:
training = train['Adj Close']
validating = valid['Adj Close']

In [10]:
model = auto_arima(training, start_p=1, start_q=1,max_p=3, max_q=3, m=12,start_P=0, seasonal=True,d=1, D=1, trace=True,error_action='ignore',suppress_warnings=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=17408.408, BIC=17440.724, Fit time=30.600 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=20640.603, BIC=20653.529, Fit time=0.866 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=19296.262, BIC=19322.115, Fit time=5.512 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=17406.930, BIC=17432.783, Fit time=27.196 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=17408.701, BIC=17441.017, Fit time=53.883 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=20642.303, BIC=20661.692, Fit time=1.369 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=17408.694, BIC=17441.010, Fit time=140.934 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=17410.288, BIC=17449.067, Fit time=86.815 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=17405.555, BIC=17424.944, Fit time=19.989 seconds
Fit

In [11]:
model.fit(training)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 0),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [12]:
forecast = model.predict(len(validating))

In [13]:
forecast = pd.DataFrame(forecast,index = valid.index,columns=['Prediction'])

In [14]:
import plotly.plotly as py
import plotly.graph_objs as go
from sklearn.metrics import mean_squared_error

In [15]:
rmse = np.sqrt(mean_squared_error(validating, forecast))
rmse

6.460567246946571

In [16]:
trace0 = go.Scatter(x = training.index, y=training, mode = 'lines', name ='Train')
trace1 = go.Scatter(x = validating.index, y=validating, mode = 'lines', name = 'Valid')
trace2 = go.Scatter(x = validating.index, y=forecast, mode = 'lines', name = 'ARIMA')
data = [trace0, trace1, trace2]
layout = dict(title = 'ARIMA',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Adj Close'),
              )
fig = dict(data=data, layout=layout)
py.iplot(fig, filename='ARIMA7')

D:\Programs\Anaconda\lib\site-packages\IPython\core\display.py:689: UserWarning:

Consider using IPython.display.IFrame instead

